# 16: Merge Nextbike data with VRS data

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import shapely
from shapely import Point, wkb, wkt

In [2]:
data_month_single = '7'
data_month_double = '07'

## load data

In [3]:
# load tier trip data
nextbike = pd.DataFrame(pd.read_parquet("/Volumes/T7/Master/Processed Data/"+data_month_double+"/15_nextbike_with_buffer_zones_NEU.parquet", engine="pyarrow"))
nextbike

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_location,end_location,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone
215356,21395440,2023-01-01 00:01:00,2023-01-01 00:26:00,False,False,bike,0 days 00:25:00,2.408602,2408.601747,b'\x01\x01\x00\x00\x00\xee\xcdo\x98h\x80\x1c@\...,b'\x01\x01\x00\x00\x00\xb7\xd4A^\x0fv\x1c@ca\x...,NaN,None,215356 ...,1500.0,Bonn Konrad-Adenauer-Platz,215356 POLYGON ((7.120193524138334 50.7395...
215357,21395441,2023-01-01 00:01:00,2023-01-01 00:16:00,False,False,bike,0 days 00:15:00,2.122154,2122.154442,b'\x01\x01\x00\x00\x00\xd8G\xa7\xae|V\x1c@}ZE\...,b'\x01\x01\x00\x00\x00o\xf2[t\xb2D\x1c@\xda\xa...,1194.0,Bonn Eifelstr./Macke-Haus,215356 ...,1432.0,Bonn Magdeburger Str.,215356 POLYGON ((7.120193524138334 50.7395...
215371,21413775,2023-01-04 22:56:00,2023-01-04 23:11:00,False,False,bike,0 days 00:15:00,1.117285,1117.284795,b'\x01\x01\x00\x00\x00B\x06\xf2\xec\xf2\x9d\x1...,b'\x01\x01\x00\x00\x00\xe2\x05\x11\xa9i\xa7\x1...,NaN,None,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...
215373,21413777,2023-01-04 23:01:00,2023-01-04 23:21:00,False,False,bike,0 days 00:20:00,4.812307,4812.306771,b'\x01\x01\x00\x00\x00 \x08\x90\xa1cG\x1c@\x0c...,b'\x01\x01\x00\x00\x005\xb5l\xad/r\x1c@e\xaa`T...,NaN,None,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...
215374,21413778,2023-01-04 23:01:00,2023-01-04 23:11:00,False,True,bike,0 days 00:10:00,0.756691,756.691454,b'\x01\x01\x00\x00\x002\xc6\x87\xd9\xcbf\x1c@\...,b'\x01\x01\x00\x00\x00/\xc0>:ue\x1c@\x1b\x9e^)...,686.0,Bonn Universität/Markt,215356 ...,1134.0,Bonn Wilhelm-Levison-Str.,215356 POLYGON ((7.120193524138334 50.7395...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899324,53247874,2023-12-15 15:16:00,2023-12-15 15:26:00,False,True,bike,0 days 00:10:00,1.476760,1476.759686,b'\x01\x01\x00\x00\x00\xdc.4\xd7i\x84\x1c@\xe8...,b'\x01\x01\x00\x00\x00L\xfb\xe6\xfe\xea\x81\x1...,NaN,None,215356 ...,691.0,Bonn Ollenhauerstr.,215356 POLYGON ((7.120193524138334 50.7395...
2899327,53247875,2023-12-15 15:31:00,2023-12-15 15:36:00,True,False,bike,0 days 00:05:00,0.494814,494.813946,b'\x01\x01\x00\x00\x00L\xfb\xe6\xfe\xea\x81\x1...,b'\x01\x01\x00\x00\x00\xb2\x82\xdf\x86\x18\x7f...,691.0,Bonn Ollenhauerstr.,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...
2899336,53247878,2023-12-15 17:41:00,2023-12-15 17:51:00,False,True,bike,0 days 00:10:00,0.491459,491.458771,b'\x01\x01\x00\x00\x00Q\x12\x12i\x1b\x7f\x1c@\...,b'\x01\x01\x00\x00\x00L\xfb\xe6\xfe\xea\x81\x1...,NaN,None,215356 ...,691.0,Bonn Ollenhauerstr.,215356 POLYGON ((7.120193524138334 50.7395...
2899339,53247879,2023-12-15 18:16:00,2023-12-15 18:31:00,True,False,bike,0 days 00:15:00,1.461322,1461.321822,b'\x01\x01\x00\x00\x00L\xfb\xe6\xfe\xea\x81\x1...,b'\x01\x01\x00\x00\x00\x90\x85\xe8\x108\x82\x1...,691.0,Bonn Ollenhauerstr.,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...


In [4]:
# transform geometry
point_start = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(nextbike["start_location"], crs=4326))
point_end = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(nextbike["end_location"], crs=4326))

In [5]:
# drop geometry columns in wrong format
nextbike = nextbike.drop(columns=["start_location", "end_location"])

In [6]:
# replace geometry columns with correct format
nextbike['start_location'] = point_start
nextbike['end_location'] = point_end

In [7]:
nextbike

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,start_location,end_location
215356,21395440,2023-01-01 00:01:00,2023-01-01 00:26:00,False,False,bike,0 days 00:25:00,2.408602,2408.601747,NaN,None,215356 ...,1500.0,Bonn Konrad-Adenauer-Platz,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.12540 50.75950),POINT (7.11529 50.74019)
215357,21395441,2023-01-01 00:01:00,2023-01-01 00:16:00,False,False,bike,0 days 00:15:00,2.122154,2122.154442,1194.0,Bonn Eifelstr./Macke-Haus,215356 ...,1432.0,Bonn Magdeburger Str.,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.08446 50.73756),POINT (7.06709 50.74553)
215371,21413775,2023-01-04 22:56:00,2023-01-04 23:11:00,False,False,bike,0 days 00:15:00,1.117285,1117.284795,NaN,None,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.15425 50.70325),POINT (7.16349 50.69928)
215373,21413777,2023-01-04 23:01:00,2023-01-04 23:21:00,False,False,bike,0 days 00:20:00,4.812307,4812.306771,NaN,None,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.06972 50.72477),POINT (7.11151 50.71345)
215374,21413778,2023-01-04 23:01:00,2023-01-04 23:11:00,False,True,bike,0 days 00:10:00,0.756691,756.691454,686.0,Bonn Universität/Markt,215356 ...,1134.0,Bonn Wilhelm-Levison-Str.,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.10039 50.73168),POINT (7.09908 50.72495)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899324,53247874,2023-12-15 15:16:00,2023-12-15 15:26:00,False,True,bike,0 days 00:10:00,1.476760,1476.759686,NaN,None,215356 ...,691.0,Bonn Ollenhauerstr.,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.12931 50.69743),POINT (7.12687 50.71059)
2899327,53247875,2023-12-15 15:31:00,2023-12-15 15:36:00,True,False,bike,0 days 00:05:00,0.494814,494.813946,691.0,Bonn Ollenhauerstr.,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.12687 50.71059),POINT (7.12412 50.70707)
2899336,53247878,2023-12-15 17:41:00,2023-12-15 17:51:00,False,True,bike,0 days 00:10:00,0.491459,491.458771,NaN,None,215356 ...,691.0,Bonn Ollenhauerstr.,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.12413 50.70710),POINT (7.12687 50.71059)
2899339,53247879,2023-12-15 18:16:00,2023-12-15 18:31:00,True,False,bike,0 days 00:15:00,1.461322,1461.321822,691.0,Bonn Ollenhauerstr.,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.12687 50.71059),POINT (7.12717 50.69735)


In [8]:
nextbike = gpd.GeoDataFrame(nextbike, geometry='start_location')

In [9]:
nextbike = gpd.GeoDataFrame(nextbike, geometry='end_location')

In [10]:
nextbike = gpd.GeoDataFrame(nextbike, geometry='start_location')

In [11]:
nextbike.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 771657 entries, 215356 to 2899345
Data columns (total 17 columns):
 #   Column                           Non-Null Count   Dtype          
---  ------                           --------------   -----          
 0   nextbike_trips_id                771657 non-null  int64          
 1   nextbike_trips_start_time        771657 non-null  datetime64[ns] 
 2   nextbike_trips_end_time          771657 non-null  datetime64[ns] 
 3   nextbike_trips_start_at_station  771657 non-null  bool           
 4   nextbike_trips_end_at_station    771657 non-null  bool           
 5   vehicle type                     771657 non-null  object         
 6   trip_duration                    771657 non-null  timedelta64[ns]
 7   trip_distance_kilometers         771657 non-null  float64        
 8   trip_distance_meters             771657 non-null  float64        
 9   start_stop_id                    402038 non-null  float64        
 10  start_stop_name        

In [12]:
nextbike

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,start_location,end_location
215356,21395440,2023-01-01 00:01:00,2023-01-01 00:26:00,False,False,bike,0 days 00:25:00,2.408602,2408.601747,NaN,None,215356 ...,1500.0,Bonn Konrad-Adenauer-Platz,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.12540 50.75950),POINT (7.11529 50.74019)
215357,21395441,2023-01-01 00:01:00,2023-01-01 00:16:00,False,False,bike,0 days 00:15:00,2.122154,2122.154442,1194.0,Bonn Eifelstr./Macke-Haus,215356 ...,1432.0,Bonn Magdeburger Str.,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.08446 50.73756),POINT (7.06709 50.74553)
215371,21413775,2023-01-04 22:56:00,2023-01-04 23:11:00,False,False,bike,0 days 00:15:00,1.117285,1117.284795,NaN,None,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.15425 50.70325),POINT (7.16349 50.69928)
215373,21413777,2023-01-04 23:01:00,2023-01-04 23:21:00,False,False,bike,0 days 00:20:00,4.812307,4812.306771,NaN,None,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.06972 50.72477),POINT (7.11151 50.71345)
215374,21413778,2023-01-04 23:01:00,2023-01-04 23:11:00,False,True,bike,0 days 00:10:00,0.756691,756.691454,686.0,Bonn Universität/Markt,215356 ...,1134.0,Bonn Wilhelm-Levison-Str.,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.10039 50.73168),POINT (7.09908 50.72495)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899324,53247874,2023-12-15 15:16:00,2023-12-15 15:26:00,False,True,bike,0 days 00:10:00,1.476760,1476.759686,NaN,None,215356 ...,691.0,Bonn Ollenhauerstr.,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.12931 50.69743),POINT (7.12687 50.71059)
2899327,53247875,2023-12-15 15:31:00,2023-12-15 15:36:00,True,False,bike,0 days 00:05:00,0.494814,494.813946,691.0,Bonn Ollenhauerstr.,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.12687 50.71059),POINT (7.12412 50.70707)
2899336,53247878,2023-12-15 17:41:00,2023-12-15 17:51:00,False,True,bike,0 days 00:10:00,0.491459,491.458771,NaN,None,215356 ...,691.0,Bonn Ollenhauerstr.,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.12413 50.70710),POINT (7.12687 50.71059)
2899339,53247879,2023-12-15 18:16:00,2023-12-15 18:31:00,True,False,bike,0 days 00:15:00,1.461322,1461.321822,691.0,Bonn Ollenhauerstr.,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.12687 50.71059),POINT (7.12717 50.69735)


In [13]:
#vrs_data = pd.DataFrame(pd.read_parquet("/Volumes/T7/Master/Processed Data/"+data_month_double+"/10_VRS_data_clean.parquet", engine="pyarrow"))
vrs_data = pd.DataFrame(pd.read_csv("/Volumes/T7/Master/11_data_clean.csv"))
#vrs_data = pd.DataFrame(pd.read_csv("/Volumes/T7/Master/10_VRS_data_clean.csv"))

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_1897/1296343515.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  vrs_data = pd.DataFrame(pd.read_csv("/Volumes/T7/Master/11_data_clean.csv"))


In [14]:
# transform geometry
vrs_data['station_point'] = vrs_data['station_point'].apply(wkt.loads)
vrs_data['buffer_zone'] = shapely.wkt.loads(vrs_data['buffer_zone'])
vrs_data = gpd.GeoDataFrame(vrs_data, geometry='buffer_zone')
vrs_data = gpd.GeoDataFrame(vrs_data, geometry='station_point')

In [15]:
# transform geometry
#point = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(vrs_data["station_point"], crs=4326))
#buffer = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(vrs_data["buffer_zone"], crs=4326))

In [16]:
# drop geometry columns in wrong format
#vrs_data= vrs_data.drop(columns=["station_point", "buffer_zone"])

In [17]:
# replace geometry columns with correct format
#vrs_data['station_point'] = point
#vrs_data['buffer_zone'] = buffer

In [18]:
nextbike_trips_not_nan = nextbike[nextbike['start_stop_id'].notna()]

In [19]:
nextbike_trips_not_nan

,nextbike_trips_id,nextbike_trips_start_time,nextbike_trips_end_time,nextbike_trips_start_at_station,nextbike_trips_end_at_station,vehicle type,trip_duration,trip_distance_kilometers,trip_distance_meters,start_stop_id,start_stop_name,start_buffer_zone,end_stop_id,end_stop_name,end_buffer_zone,start_location,end_location
215357,21395441,2023-01-01 00:01:00,2023-01-01 00:16:00,False,False,bike,0 days 00:15:00,2.122154,2122.154442,1194.0,Bonn Eifelstr./Macke-Haus,215356 ...,1432.0,Bonn Magdeburger Str.,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.08446 50.73756),POINT (7.06709 50.74553)
215374,21413778,2023-01-04 23:01:00,2023-01-04 23:11:00,False,True,bike,0 days 00:10:00,0.756691,756.691454,686.0,Bonn Universität/Markt,215356 ...,1134.0,Bonn Wilhelm-Levison-Str.,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.10039 50.73168),POINT (7.09908 50.72495)
215388,21413792,2023-01-04 23:01:00,2023-01-04 23:11:00,False,True,bike,0 days 00:10:00,0.596288,596.287809,1102.0,Bonn Friedensplatz,215356 ...,1103.0,Bonn Markt,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.09742 50.73665),POINT (7.10250 50.73492)
215394,21413798,2023-01-04 23:06:00,2023-01-04 23:16:00,False,False,bike,0 days 00:10:00,0.843076,843.075548,1102.0,Bonn Friedensplatz,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.09750 50.73649),POINT (7.09646 50.74405)
215404,21413807,2023-01-04 23:11:00,2023-01-04 23:31:00,False,False,bike,0 days 00:20:00,1.749380,1749.380473,1140.0,Bonn Mozartstr.,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.08697 50.73343),POINT (7.07257 50.72707)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899189,53247837,2023-12-12 06:51:00,2023-12-12 07:01:00,True,False,bike,0 days 00:10:00,1.345842,1345.841787,161.0,Bonn Bad Godesberg Bf,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.15918 50.68314),POINT (7.15722 50.69518)
2899262,53247857,2023-12-13 21:01:00,2023-12-13 21:11:00,False,False,bike,0 days 00:10:00,1.159259,1159.259363,1131.0,Bonn Weberstr.,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.10180 50.72589),POINT (7.11064 50.72030)
2899282,53247866,2023-12-14 11:51:00,2023-12-14 12:01:00,False,False,bike,0 days 00:10:00,0.860385,860.385079,698.0,Bonn Max-Löbner-Str./Friesdorf,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.13793 50.70071),POINT (7.13019 50.70055)
2899327,53247875,2023-12-15 15:31:00,2023-12-15 15:36:00,True,False,bike,0 days 00:05:00,0.494814,494.813946,691.0,Bonn Ollenhauerstr.,215356 ...,NaN,None,215356 POLYGON ((7.120193524138334 50.7395...,POINT (7.12687 50.71059),POINT (7.12412 50.70707)


In [20]:
vrs = vrs_data.copy()

In [21]:
vrs['actual_arrival_time'] =  pd.to_datetime(vrs['actual_arrival_time'])
vrs['actual_departure_time'] =  pd.to_datetime(vrs['actual_departure_time'])
vrs['scheduled_arrival_time'] =  pd.to_datetime(vrs['scheduled_arrival_time'])
vrs['scheduled_departure_time'] =  pd.to_datetime(vrs['scheduled_departure_time'])

In [22]:
# Assuming you have a dataframe called 'df'

# Slice the dataframe into three equal parts
num_rows = len(vrs)
part_size = num_rows // 3

# First part
part1 = vrs.iloc[:part_size]

# Second part
part2 = vrs.iloc[part_size:2*part_size]

# Third part
part3 = vrs.iloc[2*part_size:]

In [23]:
# Create a new column in vrs_data to store the count
part1['nextbike_trips_count'] = 0
part1['nextbike_trips_id'] = None
part1['nextbike_trips_end_at_station_count'] = 0
part1['nextbike_trips_end_at_station_id'] = None

# Iterate over each row in vrs_data
for index, row in part1.iterrows():
    # Get the stop_id and scheduled_arrival_time
    stop_id = row['stop_id']
    scheduled_arrival_time = row['scheduled_departure_time']
    
    # Calculate the time span
    time_span = pd.Timedelta(minutes=5)
    
    # Filter tier_trips based on stop_id and time span
    filtered_trips = nextbike_trips_not_nan[(nextbike_trips_not_nan['start_stop_id'] == stop_id) & 
                                (nextbike_trips_not_nan['nextbike_trips_start_time'] >= scheduled_arrival_time) & 
                                (nextbike_trips_not_nan['nextbike_trips_start_time'] <= scheduled_arrival_time + time_span)]
    
    filtered_ids = filtered_trips['nextbike_trips_id'].values
    
    # Count the number of entries
    count = len(filtered_ids)
    
    # Update the count in vrs_data
    part1.at[index, 'nextbike_trips_count'] = count
    part1.at[index, 'nextbike_trips_id'] = filtered_ids # list of tier_trips ids

    # Trips that ended at station
    end_trips = filtered_trips[filtered_trips['end_stop_id'].notna()]
    end_trips_ids = end_trips['nextbike_trips_id'].values
    end_trips_count = len(end_trips_ids)

    part1.at[index, 'nextbike_trips_end_at_station_count'] = end_trips_count
    part1.at[index, 'nextbike_trips_end_at_station_id'] = end_trips_ids

/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [24]:
gdf1 = gpd.GeoDataFrame(part1, geometry='station_point')

In [25]:
#gdf1 = gdf1[gdf1['nextbike_trips_count'] != 0]

In [26]:
gdf1['nextbike_trips_id'] = gdf1['nextbike_trips_id'].astype(str)

In [27]:
gdf1['nextbike_trips_id'] = gdf1['nextbike_trips_id'].str.replace('[', '').str.replace(']', '')

In [28]:
gdf1['nextbike_trips_end_at_station_id'] = gdf1['nextbike_trips_end_at_station_id'].astype(str)
gdf1['route_short_name'] = gdf1['route_short_name'].astype(str)
gdf1.to_parquet("/Volumes/T7/Master/Processed Data/"+data_month_double+"/16_VRS_Tier_Next_Merge_1_NEU_NEU_TEST.parquet", engine="pyarrow")

In [30]:
# Create a new column in vrs_data to store the count
part2['nextbike_trips_count'] = 0
part2['nextbike_trips_id'] = None
part2['nextbike_trips_end_at_station_count'] = 0
part2['nextbike_trips_end_at_station_id'] = None

# Iterate over each row in vrs_data
for index, row in part2.iterrows():
    # Get the stop_id and scheduled_arrival_time
    stop_id = row['stop_id']
    scheduled_arrival_time = row['scheduled_departure_time']
    
    # Calculate the time span
    time_span = pd.Timedelta(minutes=5)
    
    # Filter tier_trips based on stop_id and time span
    filtered_trips = nextbike_trips_not_nan[(nextbike_trips_not_nan['start_stop_id'] == stop_id) & 
                                (nextbike_trips_not_nan['nextbike_trips_start_time'] >= scheduled_arrival_time) & 
                                (nextbike_trips_not_nan['nextbike_trips_start_time'] <= scheduled_arrival_time + time_span)]
    
    filtered_ids = filtered_trips['nextbike_trips_id'].values
    
    # Count the number of entries
    count = len(filtered_ids)
    
    # Update the count in vrs_data
    part2.at[index, 'nextbike_trips_count'] = count
    part2.at[index, 'nextbike_trips_id'] = filtered_ids # list of tier_trips ids

     # Trips that ended at station
    end_trips = filtered_trips[filtered_trips['end_stop_id'].notna()]
    end_trips_ids = end_trips['nextbike_trips_id'].values
    end_trips_count = len(end_trips_ids)

    part2.at[index, 'nextbike_trips_end_at_station_count'] = end_trips_count
    part2.at[index, 'nextbike_trips_end_at_station_id'] = end_trips_ids

/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [31]:
gdf2 = gpd.GeoDataFrame(part2, geometry='station_point')

In [32]:
#gdf2 = gdf2[gdf2['nextbike_trips_count'] != 0]

In [33]:
gdf2['nextbike_trips_id'] = gdf2['nextbike_trips_id'].astype(str)

In [34]:
gdf2['nextbike_trips_id'] = gdf2['nextbike_trips_id'].str.replace('[', '').str.replace(']', '')

In [35]:
gdf2

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,...,arrival_delay,departure_delay,service,date,weekday,transfer_stop,nextbike_trips_count,nextbike_trips_id,nextbike_trips_end_at_station_count,nextbike_trips_end_at_station_id
291330,14808494,14808494,14808494,602,6,602,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,...,NaN,NaN,1,2023-07-01,Saturday,True,0,,0,[]
291331,14808890,14808890,14808890,602,6,602,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,...,NaN,NaN,1,2023-07-03,Monday,True,0,,0,[]
291332,14810042,14810042,14810042,602,6,602,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,...,0 days 00:00:15,0 days 00:00:30,1,2023-07-04,Tuesday,True,0,,0,[]
291333,14811158,14811158,14811158,602,6,602,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,...,0 days 00:00:15,0 days 00:00:30,1,2023-07-05,Wednesday,True,0,,0,[]
291334,14812382,14812382,14812382,602,6,602,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,43,...,0 days 00:00:15,0 days 00:00:30,1,2023-07-06,Thursday,True,0,,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582655,20666968,20666968,20666968,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,...,0 days 00:00:15,0 days 00:00:30,1,2023-07-03,Monday,False,0,,0,[]
582656,20668606,20668606,20668606,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,...,0 days 00:00:15,0 days 00:00:30,1,2023-07-04,Tuesday,False,0,,0,[]
582657,20669068,20669068,20669068,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,...,0 days 00:00:15,0 days 00:00:30,1,2023-07-05,Wednesday,False,0,,0,[]
582658,20670664,20670664,20670664,604,6,604,3,Bus,SWB Stadtwerke Bonn Verkehrs GmbH,64,...,0 days 00:00:15,0 days 00:00:30,1,2023-07-06,Thursday,False,0,,0,[]


In [ ]:
gdf2['nextbike_trips_end_at_station_id'] = gdf2['nextbike_trips_end_at_station_id'].astype(str)
gdf2['route_short_name'] = gdf2['route_short_name'].astype(str)
gdf2.to_parquet("/Volumes/T7/Master/Processed Data/"+data_month_double+"/16_VRS_Tier_Next_Merge_2_NEU_NEU_TEST.parquet", engine="pyarrow")

In [36]:
# Create a new column in vrs_data to store the count
part3['nextbike_trips_count'] = 0
part3['nextbike_trips_id'] = None
part3['nextbike_trips_end_at_station_count'] = 0
part3['nextbike_trips_end_at_station_id'] = None

# Iterate over each row in vrs_data
for index, row in part3.iterrows():
    # Get the stop_id and scheduled_arrival_time
    stop_id = row['stop_id']
    scheduled_arrival_time = row['scheduled_departure_time']
    
    # Calculate the time span
    time_span = pd.Timedelta(minutes=5)
    
    # Filter tier_trips based on stop_id and time span
    filtered_trips = nextbike_trips_not_nan[(nextbike_trips_not_nan['start_stop_id'] == stop_id) & 
                                (nextbike_trips_not_nan['nextbike_trips_start_time'] >= scheduled_arrival_time) & 
                                (nextbike_trips_not_nan['nextbike_trips_start_time'] <= scheduled_arrival_time + time_span)]
    
    filtered_ids = filtered_trips['nextbike_trips_id'].values
    
    # Count the number of entries
    count = len(filtered_ids)
    
    # Update the count in vrs_data
    part3.at[index, 'nextbike_trips_count'] = count
    part3.at[index, 'nextbike_trips_id'] = filtered_ids # list of tier_trips ids

     # Trips that ended at station
    end_trips = filtered_trips[filtered_trips['end_stop_id'].notna()]
    end_trips_ids = end_trips['nextbike_trips_id'].values
    end_trips_count = len(end_trips_ids)

    part3.at[index, 'nextbike_trips_end_at_station_count'] = end_trips_count
    part3.at[index, 'nextbike_trips_end_at_station_id'] = end_trips_ids

/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/miniconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [37]:
gdf3 = gpd.GeoDataFrame(part3, geometry='station_point')

In [ ]:
#gdf3 = gdf3[gdf3['nextbike_trips_count'] != 0]

In [38]:
gdf3['nextbike_trips_id'] = gdf3['nextbike_trips_id'].astype(str)

In [39]:
gdf3['nextbike_trips_id'] = gdf3['nextbike_trips_id'].str.replace('[', '').str.replace(']', '')

In [40]:
gdf3['nextbike_trips_end_at_station_id'] = gdf3['nextbike_trips_end_at_station_id'].astype(str)
gdf3['route_short_name'] = gdf3['route_short_name'].astype(str)
gdf3.to_parquet("/Volumes/T7/Master/Processed Data/"+data_month_double+"/16_VRS_Tier_Next_Merge_3_NEU_NEU_TEST.parquet", engine="pyarrow")

In [41]:
combined = pd.concat([gdf1, gdf2, gdf3])

In [45]:
combined['route_short_name'].unique()

array(['18', 'SB55', '16', '540', '600', 'SB60', '601', '602', 602, 603,
       604, '604', '63', '65', '66', '68'], dtype=object)

In [47]:
combined['route_short_name'] = combined['route_short_name'].astype(str)
combined['nextbike_trips_end_at_station_id'] = combined['nextbike_trips_end_at_station_id'].astype(str)
combined.to_parquet("/Volumes/T7/Master/Processed Data/"+data_month_double+"/16_VRS_Next_Merge_Complete_NEU_NEU_TEST.parquet", engine="pyarrow")